In [1]:
import pandas as pd
import numpy as np
import requests
from requests import Request
import json
import csv
import dotenv
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()
dotenv.load_dotenv('/Users/abc/Documents/Workspace/misc/.env')
api_key = os.environ["DUNE_API_KEY"]
headers = {"X-Dune-API-Key": api_key}

1. Search for days where reversed traces are possibly included in the calculation of daily buffer amounts:
- Download the query results from Dune and Flipside as CSV files
- Read the CSV files
- Clean and prepare the data
- Analyze the data
- Save the results

In [3]:
# Downloading query result from Dune
# execute the query 
query_id = 3949841 #Dune query to check daily amounts
base_url = f"https://api.dune.com/api/v1/query/{query_id}/execute"
params = {
    "performance": "large",
}
result_response = requests.request("POST", base_url, headers=headers, params=params)
result_data = result_response.json()
execution_id = result_data.get("execution_id")
print(execution_id)

01JEBJRQYMYGH0CKAR9H63C00G


In [6]:
# check execution status
execution_status_url = f"https://api.dune.com/api/v1/execution/{execution_id}/status"
status_response = requests.get(execution_status_url, headers=headers)
status_data = status_response.json()
state = status_data.get("state")
print(state)

QUERY_STATE_COMPLETED


In [7]:
# download the query result
base_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results/csv"
query_result_csv = requests.request("GET", base_url, headers=headers)
with open("dune_amounts_51224.csv", "wb") as f: # change here the file name
    f.write(query_result_csv.content)

To download the flipside_amount_date.csv:
- run the query on Flipside and save result as CSV file 

In [8]:

# Load daily amounts from Dune and Flipside
dune_data = pd.read_csv('dune_amounts_51224.csv')
flipside_data = pd.read_csv('flipside_amount_51224.csv')
flipside_data.columns = flipside_data.columns.str.lower()
dune_data['time'] = pd.to_datetime(dune_data['time']).dt.date
flipside_data['time'] = flipside_data['time'].str.strip()# Clean the TIME column by stripping whitespace
flipside_data['time'] = pd.to_datetime(flipside_data['time'],errors='coerce', format='%Y-%m-%d %H:%M:%S.%f') # Convert TIME column to datetime with error handling
flipside_data = flipside_data.dropna(subset=['time'])# Drop rows with invalid dates (if any)
flipside_data['time'] = flipside_data['time'].dt.date

# Merge data on date
merged_data = pd.merge(dune_data, flipside_data, on='time', suffixes=('_dune', '_flipside'))
merged_data['amount_dune_rounded'] = merged_data['amount_dune'].round()
merged_data['amount_flipside_rounded'] = merged_data['amount_flipside'].round()

# Identify mismatched data
mismatched_data = merged_data[merged_data['amount_dune_rounded'] != merged_data['amount_flipside_rounded']]

# Save mismatched data to CSV
mismatched_data[['time', 'amount_dune', 'amount_flipside']].to_csv('mismatched_51224.csv', index=False)


05.12.2024       
There is no need to proceed with verification, as all amounts matched except for the current day, which is acceptable

2. Search for reversed hashes that were included in the calculation of daily buffer amounts:
- Download the query results from Dune and Flipside as CSV files
- Read the CSV files
- Clean and prepare the data
- Analyze the data
- Save the results

In [ ]:
# Downloading query result from Dune
# execute the query 
query_id = 1863363 # Daily comparision hashes
base_url = f"https://api.dune.com/api/v1/query/{query_id}/execute"
params = {
    "performance": "large",
}
result_response = requests.request("POST", base_url, headers=headers, params=params)
result_data = result_response.json()
execution_id = result_data.get("execution_id")
print(execution_id)

In [ ]:
# check execution status
execution_status_url = f"https://api.dune.com/api/v1/execution/{execution_id}/status"
status_response = requests.get(execution_status_url, headers=headers)
status_data = status_response.json()
state = status_data.get("state")
print(state)

In [ ]:
# download the query result
base_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results/csv"
query_result_csv = requests.request("GET", base_url, headers=headers)
with open("dune_hashes351224.csv", "wb") as f: # change here the file name
    f.write(query_result_csv.content)

To download the flipside_amount_date.csv:
- run the query on Flipside and save result as CSV file 


In [ ]:
# Load daily amounts from Dune and Flipside
dune_hashes = pd.read_csv('dune_hashes351224.csv')
# print(dune_hashes)
flipside_hashes = pd.read_csv('flipside_hashes351224.csv')
flipside_hashes.columns = flipside_hashes.columns.str.lower()
# print(flipside_hashes)
# Find unique hashes
hashes_dune = set(dune_hashes['tx_hash'])
hashes_flipside = set(flipside_hashes['tx_hash'])
# Identify hashes to check
hashes_to_check = hashes_flipside.difference(hashes_dune)
# print(hashes_to_check)
# Save hashes to check
pd.DataFrame(list(hashes_to_check), columns=['hash']).to_csv('to_check351224.csv', index=False)

Repeat this step for each day from mismached_date.csv